In [2]:
%load_ext autoreload
%autoreload 2

In [8]:
%cd /content/drive/MyDrive/data_science_project/pytorch_tutorial/2_object_detection

/content/drive/MyDrive/data_science_project/pytorch_tutorial/2_object_detection


In [1]:
import cv2  # OpenCV 라이브러리
import matplotlib.pyplot as plt 
import numpy as np
import torch

%matplotlib inline

In [3]:
def ssd_predict(img_index, img_list, dataset, net=None, dataconfidence_level=0.5):
    """
    SSD로 예측하는 함수

    Parameters
    ----------
    img_index:  int
        데이터 세트 내의 예측 대상 화상의 인덱스
    img_list: list
        화상의 파일 경로 리스트
    dataset: PyTorch의 Dataset
        화상의 Dataset
    net: PyTorch의 Network
        학습시킨 SSD 네트워크
    dataconfidence_level: float
        예측에서 발견했다고 여기는 신뢰도의 임계치

    Returns
    -------
    rgb_img, true_bbox, true_label_index, predict_bbox, pre_dict_label_index, scores
    """

    # rgb의 화상 데이터를 취득
    image_file_path = img_list[img_index]
    img = cv2.imread(image_file_path)  # [높이][폭][색BGR]
    height, width, channels = img.shape  # 화상 크기 취득
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 정답 BBox를 취득
    im, gt = dataset.__getitem__(img_index)
    true_bbox = gt[:, 0:4] * [width, height, width, height]
    true_label_index = gt[:, 4].astype(int)

    # SSD로 예측
    net.eval()  # 네트워크를 추론 모드로
    x = im.unsqueeze(0)  # 미니 배치화: torch.Size([1, 3, 300, 300])
    detections = net(x)
    # detections의 형은, torch.Size([1, 21, 200, 5])  ※200은 top_k의 값

    # confidence_level이 기준 이상인 것을 꺼낸다
    predict_bbox = []
    pre_dict_label_index = []
    scores = []
    detections = detections.cpu().detach().numpy()

    # 조건 이상의 값을 추출
    find_index = np.where(detections[:, 0:, :, 0] >= dataconfidence_level)
    detections = detections[find_index]
    for i in range(len(find_index[1])):  # 추출한 물체수만큼 루프를 돈다
        if (find_index[1][i]) > 0:  # 배경 클래스가 아닌 것
            sc = detections[i][0]  # 신뢰도
            bbox = detections[i][1:] * [width, height, width, height]
            lable_ind = find_index[1][i]-1  # find_index는 미니 배치 수, 클래스, top의 tuple
            # (주석)
            # 배경 클래스는 0이므로 1을 뺀다

            # 반환값 리스트에 추가
            predict_bbox.append(bbox)
            pre_dict_label_index.append(lable_ind)
            scores.append(sc)

    return rgb_img, true_bbox, true_label_index, predict_bbox, pre_dict_label_index, scores


In [4]:
def vis_bbox(rgb_img, bbox, label_index, scores, label_names):
    """
    물체 감지의 예측 결과를 화상으로 표시하는 함수.

    Parameters
    ----------
    rgb_img:rgb의 화상
        대상 화상 데이터
    bbox: list
        물체의 BBox 리스트
    label_index: list
        물체의 라벨 인덱스
    scores: list
        물체의 신뢰도
    label_names: list
        라벨명의 배열

    Returns
    -------
    없음. rgb_img에 물체 검출 결과가 더해진 화상이 표시된다.
    """

    # 테두리 색상 설정
    num_classes = len(label_names)  # 클래스 수(배경 제외)
    colors = plt.cm.hsv(np.linspace(0, 1, num_classes)).tolist()

    # 화상 표시
    plt.figure(figsize=(10, 10))
    plt.imshow(rgb_img)
    currentAxis = plt.gca()

    # BBox만큼 루프
    for i, bb in enumerate(bbox):

        # 라벨명
        label_name = label_names[label_index[i]]
        color = colors[label_index[i]]  # 클래스마다 다른 색깔의 테두리를 부여

        # 테두리에 붙이는 라벨 (예: person: 0.72)
        if scores is not None:
            sc = scores[i]
            display_txt = '%s: %.2f' % (label_name, sc)
        else:
            display_txt = '%s: ans' % (label_name)

        # 테두리의 좌표
        xy = (bb[0], bb[1])
        width = bb[2] - bb[0]
        height = bb[3] - bb[1]

        # 직사각형 그리기
        currentAxis.add_patch(plt.Rectangle(
            xy, width, height, fill=False, edgecolor=color, linewidth=2))

        # 직사각형의 테두리의 좌측 상단에 라벨을 그린다
        currentAxis.text(xy[0], xy[1], display_txt, bbox={
                         'facecolor': color, 'alpha': 0.5})


In [5]:
class SSDPredictShow():
    """SSD의 예측과 화상의 표시를 한 번에 수행하는 클래스"""

    def __init__(self, img_list, dataset,  eval_categories, net=None, dataconfidence_level=0.6):
        self.img_list = img_list
        self.dataset = dataset
        self.net = net
        self.dataconfidence_level = dataconfidence_level
        self.eval_categories = eval_categories

    def show(self, img_index, predict_or_ans):
        """
        물체 감지의 예측 결과를 표시하는 함수.

        Parameters
        ----------
        img_index:  int
            데이터 세트 내의 예측 대상 화상의 인덱스.
        predict_or_ans: text
            'precit', 'ans'에서, BBox의 예측과 정답 중 어느 것을 표시할지 지정

        Returns
        -------
        없음. rgb_img에 물체 검출 결과가 더해진 화상이 표시된다.
        """
        rgb_img, true_bbox, true_label_index, predict_bbox, pre_dict_label_index, scores = ssd_predict(img_index, self.img_list,
                                                                 self.dataset,
                                                                 self.net,
                                                                 self.dataconfidence_level)

        if predict_or_ans == "predict":
            vis_bbox(rgb_img, bbox=predict_bbox, label_index=pre_dict_label_index,
                     scores=scores, label_names=self.eval_categories)

        elif predict_or_ans == "ans":
            vis_bbox(rgb_img, bbox=true_bbox, label_index=true_label_index,
                     scores=None, label_names=self.eval_categories)


In [9]:
from utils.ssd_model import make_datapath_list, VOCDataset, DataTransform, Anno_xml2list, od_collate_fn


# 파일 경로 리스트를 취득
rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath)

# Dataset을 작성
voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']
color_mean = (104, 117, 123)  # (BGR)의 색 평균값
input_size = 300  # 화상의 input 크기를 300×300로 한다

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="val", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))



In [13]:
from utils.ssd_model import SSD

# SSD300の設定
ssd_cfg = {
    'num_classes': 21,  # 배경 클래스를 포함한 총 클래스 수
    'input_size': 300,  # 화상의 입력 크기
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],  # 출력할 Box 화면비의 종류
    'feature_maps': [38, 19, 10, 5, 3, 1],  # 각 source의 화상 크기
    'steps': [8, 16, 32, 64, 100, 300],  # DBOX의 크기를 정한다
    'min_sizes': [30, 60, 111, 162, 213, 264],  # DBOX의 크기를 정한다
    'max_sizes': [60, 111, 162, 213, 264, 315],  # DBOX의 크기를 정한다
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

# SSD 네트워크 모델
net = SSD(phase="inference", cfg=ssd_cfg)
net.eval()

# SSD의 학습된 가중치를 설정
# net_weights = torch.load('./weights/ssd300_50.pth',
#                          map_location={'cuda:0': 'cpu'})

net_weights = torch.load('./weights/ssd300_mAP_77.43_v2.pth',
                        map_location={'cuda:0': 'cpu'})

net.load_state_dict(net_weights)

# GPU를 사용할 수 있는지 확인
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("사용 중인 장치:", device)

print('네트워크 설정 완료: 학습된 가중치를 로드했습니다')


사용 중인 장치: cpu
네트워크 설정 완료: 학습된 가중치를 로드했습니다


In [12]:
# 결과 그리기
ssd = SSDPredictShow(img_list=train_img_list, dataset=train_dataset, eval_categories=voc_classes,
                     net=net, dataconfidence_level=0.6)
img_index = 0
ssd.show(img_index, "predict")
ssd.show(img_index, "ans")

Output hidden; open in https://colab.research.google.com to view.